In [ ]:
# UCSB solver cell: analytic Φ + single/parallel + Matryoshka
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Optional, Tuple, Any, List
from concurrent.futures import ProcessPoolExecutor, as_completed
import time

from scipy.fft import fft2, ifft2
from scipy.interpolate import interp1d

# ---- Helper: build V_t grid (X-mask) ----

def build_vt_grid(N: int, x_min_nm: float, x_max_nm: float, y_min_nm: float, y_max_nm: float,
                  bar_width_nm: float, V_NS: float, V_EW: float) -> np.ndarray:
    x_nm = np.linspace(x_min_nm, x_max_nm, N)
    y_nm = np.linspace(y_min_nm, y_max_nm, N)
    X_nm, Y_nm = np.meshgrid(x_nm, y_nm, indexing="ij")
    sqrt2 = np.sqrt(2.0)
    d1 = np.abs(Y_nm - X_nm) / sqrt2
    d2 = np.abs(Y_nm + X_nm) / sqrt2
    bar_mask = (d1 <= bar_width_nm/2.0) | (d2 <= bar_width_nm/2.0)
    remain = ~bar_mask
    ns_region = remain & (np.abs(Y_nm) > np.abs(X_nm))
    ew_region = remain & (np.abs(X_nm) > np.abs(Y_nm))
    Vt = np.zeros((N, N), dtype=float)
    Vt[ns_region] = V_NS
    Vt[ew_region] = V_EW
    return Vt

# ---- UCSB config/solver ----

@dataclass
class UCSBConfig:
    # constants
    e: float = 1.602e-19
    epsilon_0: float = 8.854e-12
    h: float = 6.62607015e-34
    # material
    B: float = 13.0
    dt: float = 30e-9
    db: float = 30e-9
    epsilon_perp: float = 3.0
    epsilon_parallel: float = 6.6
    # grid (nm)
    Nx: int = 64
    Ny: int = 64
    x_min_nm: float = -150.0
    x_max_nm: float = 150.0
    y_min_nm: float = -150.0
    y_max_nm: float = 150.0
    # gates
    V_B: float = 0.0
    Vt_grid: Optional[np.ndarray] = None
    # energies
    exc_file: str = "data/0-data/Exc_data_new.csv"
    exc_scale: float = 1.0
    # optimiser
    niter: int = 10
    step_size: float = 1.0
    lbfgs_maxiter: int = 1000
    lbfgs_maxfun: int = 2_000_000

class UCSBSolver:
    J_to_meV = 1.0/(1.602e-22)
    def __init__(self, cfg: UCSBConfig):
        self.cfg = cfg
        self.Nx = int(cfg.Nx); self.Ny = int(cfg.Ny)
        # grid
        self.x_min = cfg.x_min_nm*1e-9; self.x_max = cfg.x_max_nm*1e-9
        self.y_min = cfg.y_min_nm*1e-9; self.y_max = cfg.y_max_nm*1e-9
        self.Lx = self.x_max - self.x_min; self.Ly = self.y_max - self.y_min
        self.dx = self.Lx/self.Nx; self.dy = self.Ly/self.Ny
        self.dA = self.dx*self.dy; self.A_total = self.Lx*self.Ly
        self.x = np.linspace(self.x_min, self.x_max, self.Nx)
        self.y = np.linspace(self.y_min, self.y_max, self.Ny)
        # Φ from Vt by screening relation + uniform back-gate offset
        Vt = np.zeros((self.Nx,self.Ny)) if cfg.Vt_grid is None else np.asarray(cfg.Vt_grid)
        kx = 2*np.pi*np.fft.fftfreq(self.Nx, d=self.dx)
        ky = 2*np.pi*np.fft.fftfreq(self.Ny, d=self.dy)
        KX, KY = np.meshgrid(kx, ky, indexing="ij"); q = np.sqrt(KX**2+KY**2)
        beta = np.sqrt(cfg.epsilon_parallel/cfg.epsilon_perp)
        denom = np.sinh(beta*(cfg.dt+cfg.db)*q); numer = np.sinh(beta*cfg.db*q)
        denom[denom==0] = 1e-20
        Rq = numer/denom
        Phi = np.real(ifft2(fft2(Vt)*Rq)) + cfg.V_B*cfg.dt/(cfg.dt+cfg.db)
        self.Phi = Phi
        # kernels
        self.D = cfg.e*cfg.B/cfg.h
        self.ell_B = np.sqrt(cfg.h/(2*np.pi*cfg.e*cfg.B))
        kx = 2*np.pi*np.fft.fftfreq(self.Nx, d=self.dx)
        ky = 2*np.pi*np.fft.fftfreq(self.Ny, d=self.dy)
        KX, KY = np.meshgrid(kx, ky, indexing="ij"); q = np.sqrt(KX**2+KY**2); q[0,0]=1e-20
        self.G_q = np.exp(-0.5*(self.ell_B*q)**2)
        eps_h = np.sqrt(cfg.epsilon_perp*cfg.epsilon_parallel)
        self.Vq = (cfg.e**2/(4*np.pi*cfg.epsilon_0*eps_h))*(4*np.pi*np.sinh(beta*cfg.dt*q)*np.sinh(beta*cfg.db*q))/(np.sinh(beta*(cfg.dt+cfg.db)*q)*q)
        # Exc
        data = np.loadtxt(cfg.exc_file, delimiter=",", skiprows=1)
        self.exc = interp1d(data[:,0], data[:,1], kind="linear", bounds_error=False, fill_value="extrapolate")
        # init ν
        self.nu0 = np.clip(0.5 - 1.0*(self.Phi - np.median(self.Phi)), 0.0, 1.0).flatten()
    def gaussian(self, arr: np.ndarray)->np.ndarray:
        return np.real(ifft2(fft2(arr)*self.G_q))
    def energy(self, nu_flat: np.ndarray)->float:
        nu = nu_flat.reshape((self.Nx,self.Ny))
        nu_eff = self.gaussian(nu); n_eff = nu_eff*self.D
        n_q = fft2(n_eff)*self.dA
        E_C = 0.5/self.A_total*np.sum(self.Vq*np.abs(n_q)**2)*self.J_to_meV
        E_phi = np.sum(-self.cfg.e*self.Phi*n_eff)*self.dA*self.J_to_meV
        E_xc = float(np.sum(self.exc(nu_eff))*self.dA*self.D)*float(self.cfg.exc_scale)
        return float(E_phi+E_xc+E_C)
    def optimise(self):
        from scipy.optimize import basinhopping
        self.energy_history=[self.energy(self.nu0.copy())]
        def cb(x,f,acc):
            if acc: self.energy_history.append(float(f))
        bounds=[(0.0,1.0)]*(self.Nx*self.Ny)
        res=basinhopping(self.energy, self.nu0.copy(), minimizer_kwargs={
            "method":"L-BFGS-B","bounds":bounds,
            "options":{"maxiter":self.cfg.lbfgs_maxiter,"maxfun":self.cfg.lbfgs_maxfun,"ftol":1e-6,"eps":1e-8}},
            niter=self.cfg.niter, stepsize=self.cfg.step_size, disp=False, callback=cb)
        self.nu_opt=res.x.reshape((self.Nx,self.Ny)); self.nu_smoothed=self.gaussian(self.nu_opt); self.optimisation_result=res
        return res
    def plot_results(self, save_dir: Path|str|None=None, *, title_extra: str="", show: bool=False):
        ext=(self.x_min/self.ell_B, self.x_max/self.ell_B, self.y_min/self.ell_B, self.y_max/self.ell_B)
        out=Path(save_dir) if save_dir else None
        if out: out.mkdir(parents=True, exist_ok=True)
        fig1=plt.figure(figsize=(6,5)); plt.imshow(self.nu_smoothed.T, extent=ext, origin="lower", cmap="inferno", aspect="auto", vmin=0, vmax=1)
        plt.title(f"Optimised Filling Factor ν(r)\nV_B={self.cfg.V_B:+.3f} V" + ("  |  "+title_extra if title_extra else ""))
        plt.xlabel("x [ℓ_B]"); plt.ylabel("y [ℓ_B]"); plt.colorbar(label="ν"); plt.tight_layout()
        if out: fig1.savefig(Path(out)/"nu_smoothed.png", dpi=300)
        fig2=plt.figure(figsize=(6,5)); plt.imshow(self.Phi.T, extent=ext, origin="lower", cmap="viridis", aspect="auto")
        plt.title("External Potential Φ(r) [V]" + ("\n"+title_extra if title_extra else ""))
        plt.xlabel("x [ℓ_B]"); plt.ylabel("y [ℓ_B]"); plt.colorbar(label="Φ [V]"); plt.tight_layout()
        if out: fig2.savefig(Path(out)/"phi.png", dpi=300)
        # cross section
        yi=int(np.argmin(np.abs(self.y))); x_lB=self.x/self.ell_B
        phi_meV = (-self.cfg.e*self.Phi[:,yi])*self.J_to_meV
        fig3,ax1=plt.subplots(figsize=(6.2,4.5)); ax1.plot(x_lB, phi_meV, color="tab:red", lw=2.2, label="Φ_ext [meV]")
        ax1.set_xlabel("x [ℓ_B]"); ax1.set_ylabel("Φ_ext [meV]", color="tab:red")
        ax2=ax1.twinx(); ax2.plot(x_lB, self.nu_smoothed[:,yi], color="tab:blue", lw=2.0, label="ν"); ax2.set_ylabel("ν", color="tab:blue"); ax2.set_ylim(0,1)
        if out: fig3.savefig(Path(out)/"phi_nu_cross_section.png", dpi=300)
        if show: plt.show()
        else: plt.close(fig1); plt.close(fig2); plt.close(fig3)

# ---- Runners ----

def _magnetic_length_m(B_T: float)->float:
    h=6.62607015e-34; e=1.602e-19
    return float(np.sqrt(h/(2.0*np.pi*e*B_T)))

def _resolutions_for(N: int)->List[int]:
    if N<=32: return [N]
    res=[]; n=32
    while n<N:
        res.append(n)
        if n*2>N: res.append(N); break
        n*=2
    return res


def run_ucsb_single(*,
    V_NS: float, V_EW: float, V_B: float,
    B_T: float = 13.0,
    N: int = 64,
    bar_width_nm: float = 35.0,
    dt: float = 30e-9,
    db: float = 30e-9,
    xc_scale: float = 1.0,
    niter: int = 10,
    step_size: float = 1.0,
    lbfgs_maxiter: int = 1000,
    lbfgs_maxfun: int = 2_000_000,
    matryoshka: bool = False,
    coarse_accept_limit: int = 3,
    label: str = "UCSB_single",
    ) -> Path:
    # domain in ℓ_B
    l_B=_magnetic_length_m(B_T)
    HALF=20.0*l_B*1e9
    out_dir = Path("analysis_folder")/Path(label)
    out_dir.parent.mkdir(parents=True, exist_ok=True)
    out_dir.mkdir(parents=True, exist_ok=True)

    def build_solver(res: int)->UCSBSolver:
        vt = build_vt_grid(res, -HALF, HALF, -HALF, HALF, bar_width_nm, V_NS, V_EW)
        cfg = UCSBConfig(Nx=res, Ny=res, B=B_T, dt=dt, db=db,
                         x_min_nm=-HALF, x_max_nm=HALF, y_min_nm=-HALF, y_max_nm=HALF,
                         V_B=V_B, Vt_grid=vt, exc_scale=xc_scale,
                         niter=niter, step_size=step_size,
                         lbfgs_maxiter=lbfgs_maxiter, lbfgs_maxfun=lbfgs_maxfun)
        return UCSBSolver(cfg)

    t0=time.time()
    if not matryoshka:
        solver = build_solver(N)
        solver.optimise()
    else:
        # progressive refinement
        prev_solver=None
        for res in _resolutions_for(N):
            solver = build_solver(res)
            if prev_solver is not None:
                # upsample previous ν to new res using FFT zero-padding-like bilinear
                src=np.linspace(0.0,1.0,prev_solver.Nx); tgt=np.linspace(0.0,1.0,res)
                from scipy.interpolate import RegularGridInterpolator
                rgi=RegularGridInterpolator((src,src), prev_solver.nu_opt, bounds_error=False, fill_value=None)
                X,Y=np.meshgrid(tgt,tgt, indexing="ij"); solver.nu0=rgi(np.stack([X,Y],-1)).flatten()
            # early-stop logic
            from scipy.optimize import basinhopping
            accepted=0; best_x=solver.nu0.copy(); best_E=solver.energy(solver.nu0.copy())
            def cb(x,f,acc):
                nonlocal accepted,best_x
                if acc:
                    accepted+=1; best_x=x.copy()
                    if res<N and coarse_accept_limit>0 and accepted>=coarse_accept_limit:
                        raise RuntimeError("EARLY_STOP")
            try:
                basinhopping(solver.energy, solver.nu0.copy(), minimizer_kwargs={
                    "method":"L-BFGS-B","bounds":[(0.0,1.0)]*(res*res),
                    "options":{"maxiter":lbfgs_maxiter,"maxfun":lbfgs_maxfun,"ftol":1e-6,"eps":1e-8}},
                    niter=niter, stepsize=0.01, disp=False, callback=cb)
                solver.nu_opt = best_x.reshape((res,res))
                solver.nu_smoothed = solver.gaussian(solver.nu_opt)
            except RuntimeError:
                solver.nu_opt = best_x.reshape((res,res))
                solver.nu_smoothed = solver.gaussian(solver.nu_opt)
            prev_solver=solver
    exec_sec=time.time()-t0

    title=f"V_NS={V_NS:+.2f} V, V_EW={V_EW:+.2f} V"
    solver.plot_results(save_dir=str(out_dir), title_extra=title, show=False)

    # Save arrays and config
    np.savez_compressed(out_dir/"results.npz", nu_opt=solver.nu_opt, nu_smoothed=solver.nu_smoothed,
                        Phi=solver.Phi, x=solver.x, y=solver.y)
    with (out_dir/"simulation_parameters.txt").open("w", encoding="utf-8") as f:
        for k,v in asdict(solver.cfg).items(): f.write(f"{k} = {v}\n")
        f.write("\n# Execution time\n"); f.write(f"execution_time_seconds = {exec_sec:.6f}\n"); f.write(f"execution_time_minutes = {exec_sec/60:.6f}\n")
    return out_dir


def run_ucsb_parallel(V_NS_EW_PAIRS: List[Tuple[float,float]], *,
    V_B: float,
    B_T: float = 13.0,
    N: int = 64,
    bar_width_nm: float = 35.0,
    dt: float = 30e-9,
    db: float = 30e-9,
    xc_scale: float = 1.0,
    niter: int = 10,
    step_size: float = 1.0,
    lbfgs_maxiter: int = 1000,
    lbfgs_maxfun: int = 2_000_000,
    matryoshka: bool = False,
    coarse_accept_limit: int = 3,
    label: str = "UCSB_parallel",
    max_workers: Optional[int] = None,
    ) -> Path:
    out_dir = Path("analysis_folder")/Path(label)
    out_dir.parent.mkdir(parents=True, exist_ok=True)
    out_dir.mkdir(parents=True, exist_ok=True)

    def _task(pair: Tuple[float,float]):
        sub = out_dir / (f"VNS_{pair[0]:+0.2f}_VEW_{pair[1]:+0.2f}")
        return run_ucsb_single(V_NS=pair[0], V_EW=pair[1], V_B=V_B, B_T=B_T, N=N,
                               bar_width_nm=bar_width_nm, dt=dt, db=db, xc_scale=xc_scale,
                               niter=niter, step_size=step_size,
                               lbfgs_maxiter=lbfgs_maxiter, lbfgs_maxfun=lbfgs_maxfun,
                               matryoshka=matryoshka, coarse_accept_limit=coarse_accept_limit,
                               label=str(sub))

    if len(V_NS_EW_PAIRS)==1:
        _task(V_NS_EW_PAIRS[0])
    else:
        with ProcessPoolExecutor(max_workers=max_workers or min(4,len(V_NS_EW_PAIRS))) as ex:
            futs=[ex.submit(_task, pair) for pair in V_NS_EW_PAIRS]
            for fut in as_completed(futs): fut.result()
    return out_dir


In [ ]:
# Main 1/4: Single, Matryoshka OFF（UCSB）
# 調整パラメータ
B_FIELD_T = 13.0
GRID_N = 64
BAR_WIDTH_NM = 35.0
V_NS, V_EW = (0.19, 0.51)
V_B = -0.19
NITER = 10
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2_000_000
XC_SCALE = 1.8
LABEL = "UCSB_single_off"

out_dir = run_ucsb_single(
    V_NS=V_NS, V_EW=V_EW, V_B=V_B,
    B_T=B_FIELD_T, N=GRID_N,
    bar_width_nm=BAR_WIDTH_NM,
    xc_scale=XC_SCALE,
    niter=NITER, lbfgs_maxiter=LBFGS_MAXITER, lbfgs_maxfun=LBFGS_MAXFUN,
    matryoshka=False,
    label=LABEL,
)
print(f"Saved to {out_dir}")


In [ ]:
# Main 2/4: Single, Matryoshka ON（UCSB）
# 調整パラメータ
B_FIELD_T = 13.0
GRID_N = 64
BAR_WIDTH_NM = 35.0
V_NS, V_EW = (0.19, 0.51)
V_B = -0.19
NITER = 10
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2_000_000
XC_SCALE = 1.8
COARSE_ACCEPT_LIMIT = 3
LABEL = "UCSB_single_on"

out_dir = run_ucsb_single(
    V_NS=V_NS, V_EW=V_EW, V_B=V_B,
    B_T=B_FIELD_T, N=GRID_N,
    bar_width_nm=BAR_WIDTH_NM,
    xc_scale=XC_SCALE,
    niter=NITER, lbfgs_maxiter=LBFGS_MAXITER, lbfgs_maxfun=LBFGS_MAXFUN,
    matryoshka=True, coarse_accept_limit=COARSE_ACCEPT_LIMIT,
    label=LABEL,
)
print(f"Saved to {out_dir}")


In [ ]:
# Main 3/4: Parallel, Matryoshka OFF（UCSB）
# 調整パラメータ
B_FIELD_T = 13.0
GRID_N = 64
BAR_WIDTH_NM = 35.0
V_B = -0.19
V_NS_EW_PAIRS = [(0.10, 0.60), (0.19, 0.51), (0.30, 0.70)]
NITER = 10
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2_000_000
XC_SCALE = 1.8
LABEL = "UCSB_parallel_off"
MAX_WORKERS = None  # 自動

out_dir = run_ucsb_parallel(
    V_NS_EW_PAIRS,
    V_B=V_B,
    B_T=B_FIELD_T, N=GRID_N,
    bar_width_nm=BAR_WIDTH_NM,
    xc_scale=XC_SCALE,
    niter=NITER, lbfgs_maxiter=LBFGS_MAXITER, lbfgs_maxfun=LBFGS_MAXFUN,
    matryoshka=False,
    label=LABEL,
    max_workers=MAX_WORKERS,
)
print(f"Saved to {out_dir}")


In [ ]:
# Main 4/4: Parallel, Matryoshka ON（UCSB）
# 調整パラメータ
B_FIELD_T = 13.0
GRID_N = 64
BAR_WIDTH_NM = 35.0
V_B = -0.19
V_NS_EW_PAIRS = [(0.10, 0.60), (0.19, 0.51), (0.30, 0.70)]
NITER = 10
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2_000_000
XC_SCALE = 1.8
COARSE_ACCEPT_LIMIT = 3
LABEL = "UCSB_parallel_on"
MAX_WORKERS = None  # 自動

out_dir = run_ucsb_parallel(
    V_NS_EW_PAIRS,
    V_B=V_B,
    B_T=B_FIELD_T, N=GRID_N,
    bar_width_nm=BAR_WIDTH_NM,
    xc_scale=XC_SCALE,
    niter=NITER, lbfgs_maxiter=LBFGS_MAXITER, lbfgs_maxfun=LBFGS_MAXFUN,
    matryoshka=True, coarse_accept_limit=COARSE_ACCEPT_LIMIT,
    label=LABEL,
    max_workers=MAX_WORKERS,
)
print(f"Saved to {out_dir}")
